# 1. Second model reusing the layers through the functional syntax

## Load the data

In [1]:
import numpy as np
import tensorflow as tf

In [2]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

In [3]:
train_images = train_images.reshape((60000, 28 * 28))
train_images = train_images.astype('float32') / 255
test_images = test_images.reshape((10000, 28 * 28))
test_images = test_images.astype('float32') / 255

train_labels = tf.keras.utils.to_categorical(train_labels)
test_labels = tf.keras.utils.to_categorical(test_labels)

## Le Model

In [4]:
inputs = tf.keras.Input(shape=(784,))
first_l = tf.keras.layers.Dense(128, activation='relu')(inputs)
second_l = tf.keras.layers.Dense(10, activation='softmax')(first_l)
model = tf.keras.Model(inputs, second_l)

In [5]:
model.compile(
    optimizer="rmsprop",
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)
model.fit(train_images, train_labels, epochs=5, batch_size=64)

Epoch 1/5
938/938 [==============================] - 7s 6ms/step - loss: 0.2825 - accuracy: 0.9213
Epoch 2/5
938/938 [==============================] - 5s 5ms/step - loss: 0.1312 - accuracy: 0.9618
Epoch 3/5
938/938 [==============================] - 5s 5ms/step - loss: 0.0936 - accuracy: 0.9723
Epoch 4/5
938/938 [==============================] - 5s 5ms/step - loss: 0.0736 - accuracy: 0.9777
Epoch 5/5
938/938 [==============================] - 5s 5ms/step - loss: 0.0608 - accuracy: 0.9825


## Le Second model

In [6]:
second_model = tf.keras.Model(inputs, first_l)

Model 1's first layer output & model 2 are the same

In [7]:
model.layers[1](test_images[:1])

<tf.Tensor: shape=(1, 128), dtype=float32, numpy=
array([[0.9694876 , 2.013655  , 2.5200915 , 0.        , 0.        ,
        0.        , 0.17650075, 0.        , 0.        , 0.        ,
        0.        , 0.5180968 , 0.        , 0.        , 2.1536422 ,
        0.        , 0.18467633, 0.69022715, 0.        , 0.        ,
        0.        , 1.1668979 , 0.        , 0.        , 2.412094  ,
        0.        , 0.        , 2.1151462 , 2.1390893 , 0.7056808 ,
        0.        , 1.3823683 , 1.6759933 , 2.24496   , 0.        ,
        0.        , 2.5638027 , 3.0304947 , 0.7724538 , 0.        ,
        0.        , 0.        , 0.        , 0.21234143, 0.        ,
        0.976702  , 0.        , 0.        , 0.        , 3.329416  ,
        0.        , 1.6250241 , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.4641406 , 0.        ,
        0.        , 2.1400208 , 0.06968328, 0.        , 1.1336994 ,
        2.3812757 , 2.8873024 , 0.        , 0.        , 0.        

In [8]:
second_model(test_images[:1]).shape

TensorShape([1, 128])

The actual numbers are the same (assert_near would throw an error otherwise.).

In [9]:
tf.debugging.assert_near(
    model.layers[1](test_images[:1]),
    second_model(test_images[:1])
)

In [10]:
np.allclose(
    model.layers[1](test_images[:1]).numpy(),
    second_model(test_images[:1]).numpy()
)

True

Note that this means that you can use `model.layers[]` to run data through the net only up to a certain point.

In [11]:
model.layers

In [12]:
inputs = test_images[:1]
x = inputs
print('before:', x.shape)
for layer in model.layers[:2]:
    print('going through:', layer.name)
    x = layer(x)
print('after:', x.shape)

before: (1, 784)
going through: input_1
going through: dense
after: (1, 128)


---

# 2. Loss list with None

In [13]:
inputs = tf.keras.Input(shape=(784,))
first_l = tf.keras.layers.Dense(128, activation='relu')(inputs)
second_l = tf.keras.layers.Dense(10, activation='softmax')(first_l)
model = tf.keras.Model(inputs, [second_l, first_l]) # LIST

In [14]:
model.compile(
    optimizer="rmsprop",
    loss=["categorical_crossentropy", None], # LIST
    metrics=["accuracy"]
)
model.fit(train_images, train_labels, epochs=5, batch_size=64)

Epoch 1/5
938/938 [==============================] - 7s 6ms/step - loss: 0.2869 - dense_3_loss: 0.2869 - dense_3_accuracy: 0.9192 - dense_2_accuracy: 9.8333e-04
Epoch 2/5
938/938 [==============================] - 5s 6ms/step - loss: 0.1327 - dense_3_loss: 0.1327 - dense_3_accuracy: 0.9614 - dense_2_accuracy: 0.0032
Epoch 3/5
938/938 [==============================] - 6s 6ms/step - loss: 0.0956 - dense_3_loss: 0.0956 - dense_3_accuracy: 0.9717 - dense_2_accuracy: 0.0085
Epoch 4/5
938/938 [==============================] - 5s 6ms/step - loss: 0.0744 - dense_3_loss: 0.0744 - dense_3_accuracy: 0.9782 - dense_2_accuracy: 0.0037
Epoch 5/5
938/938 [==============================] - 5s 6ms/step - loss: 0.0620 - dense_3_loss: 0.0620 - dense_3_accuracy: 0.9817 - dense_2_accuracy: 0.0080


In [15]:
model.evaluate(test_images, test_labels) # even works on evaluate

313/313 [==============================] - 2s 5ms/step - loss: 0.0777 - dense_3_loss: 0.0777 - dense_3_accuracy: 0.9771 - dense_2_accuracy: 0.0091


[0.07774907350540161,
 0.07774907350540161,
 0.9771000146865845,
 0.009100000374019146]

In [19]:
probs, innards = model(test_images[:1])
print(probs.shape) # probs of mnist digit
print(innards.shape) # inner activations

(1, 10)
(1, 128)
